# 14. Choose, Constant, Option, Match
#### By Kwan Yin Andrew Chau

In [1]:
import nbfinder
import random
from loader import g
from loader import __
from loader import Barrier
from loader import Column
from loader import P
from loader import Pick
from loader import Pop
from loader import sydneyVertexId
from loader import hkVertexId

## choose()

__If an airport in NSW has a runway > 12,000 feet return its code else return its description__

In [2]:
g.V().hasLabel('airport').has('region','AU-NSW').choose(
    __.values('longest').is_(P.gt(12000)),
    # if greater than 12000
    __.values('code'),
    # else
    __.values('desc')
).fold().next()

['Orange Airport',
 'Moruya Airport',
 'Merimbula Airport',
 'Tamworth Airport',
 'Moree Airport',
 'Ballina Byron Gateway Airport',
 'Wagga Wagga City Airport',
 'SYD',
 'Taree Airport',
 'Newcastle Airport',
 'Dubbo City Regional Airport',
 'Parkes Airport',
 'Grafton Airport',
 'Armidale Airport',
 'Bathurst Airport',
 'Narrabri Airport',
 'Broken Hill Airport',
 'Sydney Bankstown Airport',
 'Coffs Harbour Airport',
 'Port Macquarie Airport',
 'Griffith Airport',
 'Lord Howe Island Airport',
 'Narrandera Airport',
 'Albury Airport',
 'Lismore Airport',
 'Gold Coast Airport']

__If an airport in NSW has a code of SYD or BWU report its desc else report its code__

In [3]:
g.V().hasLabel('airport').has('region','AU-NSW').choose(
    __.values('code').is_(P.within('SYD','BWU')),
    # if code is SYD or BWU
    __.values('desc'),
    __.values('code')
).fold().next()

['OAG',
 'MYA',
 'MIM',
 'TMW',
 'MRZ',
 'BNK',
 'WGA',
 'Sydney Kingsford Smith',
 'TRO',
 'NTL',
 'DBO',
 'PKE',
 'GFN',
 'ARM',
 'BHS',
 'NAA',
 'BHQ',
 'Sydney Bankstown Airport',
 'CFS',
 'PQQ',
 'GFF',
 'LDH',
 'NRA',
 'ABX',
 'LSY',
 'OOL']

## constant()

__Return "some" if the airport has less than 4 runways and "lots" if more__

In [4]:
g.V().hasLabel('airport').limit(10).choose(
    __.values('runways').is_(P.lt(4)),
    # constant values
    __.constant('some'), __.constant('lots')
).fold().next()

['lots',
 'some',
 'some',
 'some',
 'some',
 'some',
 'some',
 'some',
 'lots',
 'some']

__Return "some" or "lots" depending on whether the airport has more than 50 routes out of it__

In [5]:
g.V().hasLabel('airport').sample(10).as_('airport','routes').choose(
    __.out('route').count().is_(P.gt(50)),
    __.constant('lots'),
    __.constant('some')
).as_('lotsOrSome').select(
    'airport','routes','lotsOrSome'
).by('desc').by(__.out('route').count()).by().fold().next()

[{'airport': 'Cengiz Topel Airport', 'routes': 1, 'lotsOrSome': 'some'},
 {'airport': 'Goroka Airport', 'routes': 4, 'lotsOrSome': 'some'},
 {'airport': 'Rafha Domestic Airport', 'routes': 1, 'lotsOrSome': 'some'},
 {'airport': 'Fukuoka Airport', 'routes': 39, 'lotsOrSome': 'some'},
 {'airport': 'Siem Reap International Airport',
  'routes': 24,
  'lotsOrSome': 'some'},
 {'airport': 'Victoria Falls International Airport',
  'routes': 7,
  'lotsOrSome': 'some'},
 {'airport': 'Orlando Sanford International Airport',
  'routes': 81,
  'lotsOrSome': 'lots'},
 {'airport': 'Diqing Airport', 'routes': 5, 'lotsOrSome': 'some'},
 {'airport': 'Craig Cove Airport', 'routes': 2, 'lotsOrSome': 'some'},
 {'airport': 'Orange Airport', 'routes': 2, 'lotsOrSome': 'some'}]

_.select(values) is able to remove the keys of the results and only keep the values_

In [6]:
g.V().hasLabel('airport').limit(20).as_('a', 'b').choose(
    __.out('route').count().is_(P.gt(50)), __.constant('lots'), __.constant('some')
).as_('c').select(
    'a','b','c'
).by('desc').by(__.out('route').count()).by().select(
    # Just the values and no a,b,c
    Column.values
).fold().next()

[['Nelson Airport', 4, 'some'],
 ['Saurimo Airport', 2, 'some'],
 ['Chico Municipal Airport', 1, 'some'],
 ['Yakubu Gowon Airport', 1, 'some'],
 ['Santos Dumont Airport', 18, 'some'],
 ['Kelowna International Airport', 12, 'some'],
 ['Show Low Regional Airport', 2, 'some'],
 ['Esenboğa International Airport', 55, 'lots'],
 ['Kerikeri Airport', 1, 'some'],
 ['La Chorrera Airport', 2, 'some'],
 ['Block Island State Airport', 0, 'some'],
 ['New York John F. Kennedy International Airport', 187, 'lots'],
 ['Victoria Falls International Airport', 7, 'some'],
 ['Vinnytsia/Gavyryshivka Airport', 2, 'some'],
 ['Longjia Airport', 31, 'some'],
 ['Burketown Airport', 2, 'some'],
 ['Maya-Maya Airport', 13, 'some'],
 ['Lampedusa Airport', 4, 'some'],
 ['Hasanuddin International Airport', 27, 'some'],
 ['Whatì Airport', 1, 'some']]

## option()

__Switch like queries__

In [7]:
g.V().hasLabel('airport').sample(10).choose(
    __.values('runways')
).option(
    1, __.values('desc')
).option(
    2, __.constant('Two runways')
).option(
    # default case
    Pick.none, __.values('runways')
).fold().next()

['Ain Arnat Airport',
 'Ban Huoeisay Airport',
 'Port Augusta Airport',
 'Two runways',
 'Douala International Airport',
 'Ormoc Airport',
 4,
 'Eek Airport',
 'Southwest Florida International Airport',
 'Two runways']

## match()

__Find airports where there is a direct flight to the destination but not when returning__

In [8]:
g.V().hasLabel('airport').match(
    __.as_('origin').out().as_('destination'),
    __.not_(__.as_('destination').out().as_('origin'))
).count().next()

238

_Find airports where you can go from A→B→C but not A→C and A→B→A_

In [9]:
g.V().hasLabel('airport').match(
    __.as_('a').out().as_('b'),
    __.as_('b').out().where(P.neq('a')).as_('c'),
    __.not_(__.as_('a').out().as_('c'))
).select('a','b','c').by('code').limit(10).fold().next()

[{'a': 'NSN', 'b': 'WLG', 'c': 'BNE'},
 {'a': 'NSN', 'b': 'WLG', 'c': 'CBR'},
 {'a': 'NSN', 'b': 'WLG', 'c': 'SYD'},
 {'a': 'NSN', 'b': 'WLG', 'c': 'HLZ'},
 {'a': 'NSN', 'b': 'WLG', 'c': 'NPE'},
 {'a': 'NSN', 'b': 'WLG', 'c': 'TIU'},
 {'a': 'NSN', 'b': 'WLG', 'c': 'ZQN'},
 {'a': 'NSN', 'b': 'WLG', 'c': 'TRG'},
 {'a': 'NSN', 'b': 'WLG', 'c': 'WRE'},
 {'a': 'NSN', 'b': 'WLG', 'c': 'IVC'}]

_Find destinations from Sydney that have the same amount of runways_

In [10]:
g.V().has('code','SYD').match(
    __.as_('sydney').values('runways').as_('sydneyRunways'),
    __.as_('sydney').out('route').as_('destination').values('runways').as_('destinationRunways').where(
        'sydneyRunways', P.eq('destinationRunways')
    )
).select('destination').valueMap().select('code','runways').fold().next()

[{'code': ['YVR'], 'runways': [3]},
 {'code': ['PEK'], 'runways': [3]},
 {'code': ['KUL'], 'runways': [3]},
 {'code': ['CHC'], 'runways': [3]},
 {'code': ['LST'], 'runways': [3]},
 {'code': ['SIN'], 'runways': [3]},
 {'code': ['DEL'], 'runways': [3]},
 {'code': ['ICN'], 'runways': [3]}]

_The same can be done with multiple selects()_

In [11]:
g.V().has('code','SYD').as_('sydney').values('runways').as_('sydneyRunways').select(
    'sydney'
).out().as_('destination').values('runways').as_('destinationRunways').where(
    'sydneyRunways', P.eq('destinationRunways')
).select('destination').valueMap().select('code','runways').fold().next()

[{'code': ['YVR'], 'runways': [3]},
 {'code': ['PEK'], 'runways': [3]},
 {'code': ['KUL'], 'runways': [3]},
 {'code': ['CHC'], 'runways': [3]},
 {'code': ['LST'], 'runways': [3]},
 {'code': ['SIN'], 'runways': [3]},
 {'code': ['DEL'], 'runways': [3]},
 {'code': ['ICN'], 'runways': [3]}]

_Both the previous ways however are not as good as using a filter_

In [12]:
g.V().has('code','SYD').as_('sydney').out().as_('destination').filter(
    __.select('sydney','destination').by('runways').where('sydney', P.eq('destination'))
).valueMap('code','runways').fold().next()

[{'runways': [3], 'code': ['YVR']},
 {'runways': [3], 'code': ['PEK']},
 {'runways': [3], 'code': ['KUL']},
 {'runways': [3], 'code': ['CHC']},
 {'runways': [3], 'code': ['LST']},
 {'runways': [3], 'code': ['SIN']},
 {'runways': [3], 'code': ['DEL']},
 {'runways': [3], 'code': ['ICN']}]

_In fact you can just use where() with by()_

In [13]:
g.V().has('code', 'SYD').as_('sydney').out().where(
    P.eq('sydney')
).by('runways').valueMap('code','runways').fold().next()

[{'runways': [3], 'code': ['YVR']},
 {'runways': [3], 'code': ['PEK']},
 {'runways': [3], 'code': ['KUL']},
 {'runways': [3], 'code': ['CHC']},
 {'runways': [3], 'code': ['LST']},
 {'runways': [3], 'code': ['SIN']},
 {'runways': [3], 'code': ['DEL']},
 {'runways': [3], 'code': ['ICN']}]